# EqualWidthBinning: Feature Demonstration

This notebook demonstrates the key features of the `EqualWidthBinning` class from the binlearn library.

## Key Features Covered:
- Basic binning with different configurations
- Working with pandas DataFrames and numpy arrays
- Sklearn pipeline integration
- Parameter serialization and reconstruction
- Edge case handling

## Overview
`EqualWidthBinning` divides feature ranges into bins of equal width, making it ideal for:
- **Uniform data distributions**
- **Fast processing needs**
- **Interpretable bin boundaries**
- **Feature engineering pipelines**

## 1. Setup and Data Preparation

In [9]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from binlearn.methods import EqualWidthBinning
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [10]:
# Create sample datasets
n_samples = 200

# 1. Pandas DataFrame with mixed distributions
data = pd.DataFrame({
    'uniform': np.random.uniform(0, 100, n_samples),
    'normal': np.random.normal(50, 15, n_samples),
    'skewed': np.random.exponential(2, n_samples)
})

# 2. NumPy array
array_data = np.random.randn(n_samples, 2) * [10, 20] + [0, 50]

# 3. Classification dataset for pipeline demo
X_class, y_class = make_classification(
    n_samples=n_samples, 
    n_features=3, 
    n_informative=2, 
    n_redundant=1, 
    n_repeated=0,
    n_classes=2, 
    random_state=42
)
X_features = pd.DataFrame(X_class, columns=['feature_1', 'feature_2', 'feature_3'])

print(f"📊 Created datasets:")
print(f"   - Mixed data: {data.shape}")
print(f"   - Array data: {array_data.shape}")
print(f"   - Classification: {X_features.shape}")
print("\n📈 Data preview:")
print(data.describe().round(2))

📊 Created datasets:
   - Mixed data: (200, 3)
   - Array data: (200, 2)
   - Classification: (200, 3)

📈 Data preview:
       uniform  normal  skewed
count   200.00  200.00  200.00
mean     48.40   51.01    2.10
std      29.49   14.54    2.32
min       0.55    1.38    0.02
25%      22.86   40.02    0.52
50%      49.45   51.34    1.36
75%      75.69   60.00    2.77
max      98.69  107.79   16.34


## 2. Basic Binning Operations

In [11]:
# Basic usage with different bin counts
print("🎯 Basic EqualWidthBinning Demonstration")
print("=" * 40)

# Test with different bin configurations
for n_bins in [3, 5, 7]:
    print(f"\n📊 Using {n_bins} bins:")
    
    # Create and fit binner
    binner = EqualWidthBinning(n_bins=n_bins)
    binner.fit(data[['uniform', 'normal']])
    
    # Transform data
    binned = binner.transform(data[['uniform', 'normal']])
    
    print(f"   Bin edges (uniform): {np.round(binner.bin_edges_['uniform'], 1)}")
    print(f"   Bin edges (normal): {np.round(binner.bin_edges_['normal'], 1)}")
    print(f"   Unique bins created: {len(np.unique(binned))}")

# Demonstrate with numpy arrays
print(f"\n🔢 NumPy Array Support:")
binner_array = EqualWidthBinning(n_bins=4)
binner_array.fit(array_data)
binned_array = binner_array.transform(array_data)
print(f"   Array shape: {array_data.shape} → {binned_array.shape}")
print(f"   Unique values: {len(np.unique(binned_array))}")

🎯 Basic EqualWidthBinning Demonstration

📊 Using 3 bins:
   Bin edges (uniform): [ 0.6 33.3 66.  98.7]
   Bin edges (normal): [  1.4  36.9  72.3 107.8]
   Unique bins created: 3

📊 Using 5 bins:
   Bin edges (uniform): [ 0.6 20.2 39.8 59.4 79.1 98.7]
   Bin edges (normal): [  1.4  22.7  43.9  65.2  86.5 107.8]
   Unique bins created: 5

📊 Using 7 bins:
   Bin edges (uniform): [ 0.6 14.6 28.6 42.6 56.6 70.6 84.7 98.7]
   Bin edges (normal): [  1.4  16.6  31.8  47.   62.2  77.4  92.6 107.8]
   Unique bins created: 7

🔢 NumPy Array Support:
   Array shape: (200, 2) → (200, 2)
   Unique values: 4


In [12]:
# Visualize binning results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Original data
ax1.scatter(data['uniform'], data['normal'], alpha=0.6, color='blue')
ax1.set_title('Original Data')
ax1.set_xlabel('Uniform Distribution')
ax1.set_ylabel('Normal Distribution')

# Binned data with boundaries
binner = EqualWidthBinning(n_bins=5)
binner.fit(data[['uniform', 'normal']])
binned = binner.transform(data[['uniform', 'normal']])

scatter = ax2.scatter(data['uniform'], data['normal'], c=binned[:, 0], 
                     cmap='viridis', alpha=0.7)

# Add bin boundaries
for edge in binner.bin_edges_['uniform']:
    ax2.axvline(edge, linestyle='--', alpha=0.6, color='red', linewidth=1)
for edge in binner.bin_edges_['normal']:
    ax2.axhline(edge, linestyle='--', alpha=0.6, color='red', linewidth=1)

ax2.set_title('Binned Data (5 bins)')
ax2.set_xlabel('Uniform Distribution')
ax2.set_ylabel('Normal Distribution')
plt.colorbar(scatter, ax=ax2)

plt.tight_layout()
plt.show()

## 3. Sklearn Pipeline Integration

In [13]:
# Demonstrate pipeline integration
print("🔗 Sklearn Pipeline Integration")
print("=" * 35)

# Split data for classification
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_class, test_size=0.3, random_state=42
)

# Create pipeline with binning and classification
pipeline = Pipeline([
    ('binner', EqualWidthBinning(n_bins=5)),
    ('classifier', RandomForestClassifier(n_estimators=50, random_state=42))
])

# Fit and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = (y_pred == y_test).mean()

print(f"✅ Pipeline trained successfully")
print(f"📊 Test accuracy: {accuracy:.3f}")
print(f"🔢 Features binned: {X_train.shape[1]}")

# Show bin edges from pipeline
binner_from_pipeline = pipeline.named_steps['binner']
print(f"\n📏 Bin edges created:")
for i, (col, edges) in enumerate(binner_from_pipeline.bin_edges_.items()):
    print(f"   {col}: {np.round(edges, 2)}")

🔗 Sklearn Pipeline Integration
✅ Pipeline trained successfully
📊 Test accuracy: 0.817
🔢 Features binned: 3

📏 Bin edges created:
   feature_1: [-2.9  -1.73 -0.57  0.59  1.75  2.92]
   feature_2: [-3.09 -1.67 -0.26  1.15  2.56  3.97]
   feature_3: [-3.01 -2.09 -1.17 -0.25  0.67  1.59]


## 4. Parameter Serialization and Reconstruction

In [14]:
# Demonstrate serialization for model persistence
print("💾 Parameter Serialization & Reconstruction")
print("=" * 45)

# Create and fit original binner
original_binner = EqualWidthBinning(n_bins=6)
original_binner.fit(data)
original_result = original_binner.transform(data)

# Serialize parameters
params = original_binner.get_params()
print(f"📋 Serialized parameters: {params}")

# Reconstruct binner from parameters
reconstructed_binner = EqualWidthBinning(**params)

# Test reconstruction
reconstructed_result = reconstructed_binner.transform(data)
results_match = np.allclose(original_result, reconstructed_result)

print(f"\n🔍 Reconstruction verification:")
print(f"   ✅ Results identical: {results_match}")
print(f"   📊 Original shape: {original_result.shape}")
print(f"   📊 Reconstructed shape: {reconstructed_result.shape}")

# Test with new data
test_data = pd.DataFrame({
    'uniform': np.random.uniform(-10, 110, 50),
    'normal': np.random.normal(50, 20, 50),
    'skewed': np.random.exponential(3, 50)
})

test_result = reconstructed_binner.transform(test_data)
print(f"\n🔄 Transform new data without refitting:")
print(f"   ✅ Success: {test_result.shape}")
print(f"   🔢 Unique bins: {len(np.unique(test_result))}")

💾 Parameter Serialization & Reconstruction
📋 Serialized parameters: {'n_bins': 6, 'bin_range': None, 'clip': True, 'preserve_dataframe': False, 'fit_jointly': False, 'guidance_columns': None, 'bin_edges': {'uniform': [np.float64(0.5522117123602399), np.float64(16.90829203697549), np.float64(33.26437236159074), np.float64(49.62045268620599), np.float64(65.97653301082124), np.float64(82.33261333543649), np.float64(98.68869366005173)], 'normal': [np.float64(1.380989898963911), np.float64(19.115986975773392), np.float64(36.850984052582874), np.float64(54.58598112939236), np.float64(72.32097820620183), np.float64(90.05597528301132), np.float64(107.79097235982081)], 'skewed': [np.float64(0.022837178865083338), np.float64(2.743179514993597), np.float64(5.4635218511221115), np.float64(8.183864187250625), np.float64(10.90420652337914), np.float64(13.624548859507653), np.float64(16.344891195636166)]}, 'bin_representatives': {'uniform': [np.float64(8.730251874667864), np.float64(25.08633219928311

## 5. Edge Cases and Robustness

In [15]:
# Test edge cases
print("⚠️ Edge Case Testing")
print("=" * 25)

edge_cases = {
    'Constant values': pd.DataFrame({'const': np.full(100, 42.0)}),
    'Single value': pd.DataFrame({'single': [1.0]}),
    'Small range': pd.DataFrame({'small': np.random.uniform(0.001, 0.002, 100)}),
    'Large values': pd.DataFrame({'large': np.random.exponential(1000, 100)})
}

for case_name, case_data in edge_cases.items():
    print(f"\n🧪 Testing: {case_name}")
    try:
        binner = EqualWidthBinning(n_bins=5)
        binner.fit(case_data)
        result = binner.transform(case_data)
        
        print(f"   ✅ Success: {result.shape}")
        print(f"   📊 Unique bins: {len(np.unique(result))}")
        print(f"   📏 Range: [{case_data.iloc[:, 0].min():.6f}, {case_data.iloc[:, 0].max():.6f}]")
        
    except Exception as e:
        print(f"   ❌ Failed: {str(e)}")

⚠️ Edge Case Testing

🧪 Testing: Constant values
   ✅ Success: (100, 1)
   📊 Unique bins: 1
   📏 Range: [42.000000, 42.000000]

🧪 Testing: Single value
   ✅ Success: (1, 1)
   📊 Unique bins: 1
   📏 Range: [1.000000, 1.000000]

🧪 Testing: Small range
   ✅ Success: (100, 1)
   📊 Unique bins: 5
   📏 Range: [0.001031, 0.001996]

🧪 Testing: Large values
   ✅ Success: (100, 1)
   📊 Unique bins: 4
   📏 Range: [4.952223, 7441.722910]


## 6. Summary and Best Practices

In [16]:
# Summary and recommendations
print("📝 EqualWidthBinning Summary")
print("=" * 35)

recommendations = {
    "✅ Best for": [
        "Uniform or near-uniform distributions",
        "Fast preprocessing pipelines",
        "Interpretable bin boundaries",
        "Feature engineering workflows"
    ],
    "⚠️ Use with caution": [
        "Highly skewed distributions",
        "Data with many outliers",
        "When equal sample sizes per bin are needed"
    ],
    "💡 Tips": [
        "Start with 5-10 bins for most use cases",
        "Use get_params() for model persistence",
        "Combine with sklearn pipelines for robust workflows",
        "Handle constant features appropriately"
    ]
}

for category, items in recommendations.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  • {item}")

print(f"\n🎯 Key Features Demonstrated:")
features = [
    "Basic binning with multiple configurations",
    "Support for pandas DataFrames and numpy arrays", 
    "Seamless sklearn pipeline integration",
    "Parameter serialization and model reconstruction",
    "Robust handling of edge cases"
]

for i, feature in enumerate(features, 1):
    print(f"  {i}. {feature}")

print(f"\n✨ EqualWidthBinning demonstration complete!")

📝 EqualWidthBinning Summary

✅ Best for:
  • Uniform or near-uniform distributions
  • Fast preprocessing pipelines
  • Interpretable bin boundaries
  • Feature engineering workflows

⚠️ Use with caution:
  • Highly skewed distributions
  • Data with many outliers
  • When equal sample sizes per bin are needed

💡 Tips:
  • Start with 5-10 bins for most use cases
  • Use get_params() for model persistence
  • Combine with sklearn pipelines for robust workflows
  • Handle constant features appropriately

🎯 Key Features Demonstrated:
  1. Basic binning with multiple configurations
  2. Support for pandas DataFrames and numpy arrays
  3. Seamless sklearn pipeline integration
  4. Parameter serialization and model reconstruction
  5. Robust handling of edge cases

✨ EqualWidthBinning demonstration complete!
